In [1]:
import scanpy as sc
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from statsmodels.distributions.empirical_distribution import ECDF

In [2]:
from liana.utils.pre import _check_mat, _check_if_covered, _format_vars
from liana.resource import select_resource
from liana.resource._select_resource import _explode_complexes

1. Check matrices if csr
2. Check if | or _ is in gene names -> remove
3. Check intersect /w resource

In [3]:
groupby='label'

In [4]:
method='wilcoxon'

Load anndata and resource

In [5]:
adata = sc.read_h5ad("test.h5ad")

In [6]:
adata.X = _check_mat(adata.X, True)

Converting mat to CSR format


In [12]:
def liana_pipe(adata, groupby, resource_name, resource, verbose):
    
    # Check and Reformat Mat if needed
    adata.X = _check_mat(adata.X, verbose=verbose)
    
    # Define idents col name
    adata.obs.label = adata.obs[groupby]
    labels = adata.obs.label.cat.categories
    
    # Remove underscores from gene names
    adata.var_names = _format_vars(adata.var_names)
    
    if 'resource' not in locals():
        resource = select_resource(resource_name)
    
    # explode complexes/decomplexify
    resource = _explode_complexes(resource)
    
    # Create Entities
    entities = np.union1d(ligands, receptors)
    
    # Check overlap between resource and adata
    _check_if_covered(entities, adata.var_keys, verbose=verbose)


Load resource

In [13]:
resource = select_resource(resource_name='consensus')

In [14]:
resource = _explode_complexes(resource)

Create entities

In [15]:
entities = np.union1d(np.unique(resource["ligand"]), np.unique(resource["receptor"]))

Check overlap between resource and adata

In [16]:
_check_if_covered(entities, adata.var_keys)

In [17]:
labels = adata.obs.label.cat.categories

In [18]:
labels

Index(['B', 'CD8 T', 'NK'], dtype='object')

Process adata

In [19]:
# adata.layers['counts'] = adata.X

In [20]:
# lognorm should be the default (expected)
adata.X = adata.layers['logcounts']

In [21]:
# Get global mean for SCA before filtering
mat_mean = np.mean(adata.X)

In [22]:
# Filter to only include the relevant genes
adata = adata[:,np.intersect1d(entities, adata.var.index)]
adata

View of AnnData object with n_obs × n_vars = 90 × 553
    obs: 'label'
    uns: 'X_name', 'nCount_RNA', 'nFeature_RNA', 'orig.ident', 'seurat_annotations'
    layers: 'logcounts'

In [23]:
resource = resource[(np.isin(resource.ligand, adata.var_names)) & (np.isin(resource.receptor, adata.var_names))]

Scale for Connectome

In [24]:
adata.layers['scaled'] = sc.pp.scale(adata, copy=True).X

<ipython-input-24-f5592d25627d>:1: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  adata.layers['scaled'] = sc.pp.scale(adata, copy=True).X


In [25]:
# Only keep interactions /w complexes for which all subunits are present
temp_res = resource[['_' in x for x in resource['interaction']]].copy()
temp_res['interaction'] = temp_res['ligand_complex'] + '_' + resource['receptor_complex']
# Get those not with all subunits
temp_res = temp_res[np.logical_not([ all([x in adata.var_names for x in entity.split('_')]) for entity in temp_res.interaction])]
temp_res['interaction'] = temp_res['ligand_complex'] + '|' + resource['receptor_complex']

In [26]:
# filter them
resource = resource[~resource.interaction.isin(temp_res.interaction)]

### Stats

In [27]:
sc.tl.rank_genes_groups(adata, groupby=groupby, method=method)

In [28]:
dedict = {label:sc.get.rank_genes_groups_df(adata, label).assign(label=label).sort_values('names') for label in labels}

Calculate Mean, Sum and z-scores by group

In [29]:
# check if all is gucci
list(adata.var_names) == list(dedict['B']['names'])

True

In [30]:
for label in labels: 
    temp = adata[adata.obs.label.isin([label])].copy()
    # dedict[label]['sums'] = temp.X.sum(0)
    dedict[label]['means'] = temp.X.mean(0).A.flatten()
    dedict[label]['zscores'] = temp.layers['scaled'].mean(0)

Join Means

In [31]:
pairs = pd.DataFrame(np.array(np.meshgrid(labels, labels)).reshape(2, np.size(labels) * np.size(labels)).T).rename(columns={0: "source", 1: "target"})

In [32]:
def join_means(source, target):
    source_stats = dedict[source].copy()
    source_stats.columns = source_stats.columns.map(lambda x: 'ligand_' + str(x))
    source_stats = source_stats.rename(columns={'ligand_names':'ligand', 'ligand_label':'source'})
    
    target_stats = dedict[target].copy()
    target_stats.columns = target_stats.columns.map(lambda x: 'receptor_' + str(x))
    target_stats = target_stats.rename(columns={'receptor_names':'receptor', 'receptor_label':'target'})
    
    bound = resource.merge(source_stats).merge(target_stats)
    
    return bound

In [33]:
lr_res = pd.concat([join_means(source, target) for source, target in zip(pairs['source'], pairs['target'])])

In [34]:
lr_res['mat_mean'] = mat_mean

In [35]:
lr_res

,interaction,ligand,receptor,ligand_complex,receptor_complex,ligand_scores,ligand_logfoldchanges,ligand_pvals,ligand_pvals_adj,source,ligand_means,ligand_zscores,receptor_scores,receptor_logfoldchanges,receptor_pvals,receptor_pvals_adj,target,receptor_means,receptor_zscores,mat_mean
0,LGALS9|PTPRC,LGALS9,PTPRC,LGALS9,PTPRC,1.433668,2.440976,0.151667,0.823608,B,0.374320,0.377836,-3.440802,-1.272587,0.000580,0.008909,B,0.761533,-0.415538,0.199056
1,CD22|PTPRC,CD22,PTPRC,CD22,PTPRC,2.730388,3.604738,0.006326,0.063285,B,0.554265,0.618197,-3.440802,-1.272587,0.000580,0.008909,B,0.761533,-0.415538,0.199056
2,LGALS1|PTPRC,LGALS1,PTPRC,LGALS1,PTPRC,-2.935809,-2.024960,0.003327,0.036073,B,0.611330,-0.460597,-3.440802,-1.272587,0.000580,0.008909,B,0.761533,-0.415538,0.199056
3,LGALS9|CD44,LGALS9,CD44,LGALS9,CD44,1.433668,2.440976,0.151667,0.823608,B,0.374320,0.377836,-0.419401,-0.198308,0.674923,0.960181,B,0.655391,-0.051623,0.199056
4,ADAM10|CD44,ADAM10,CD44,ADAM10,CD44,0.089872,-0.020321,0.928389,0.974192,B,0.119797,-0.002707,-0.419401,-0.198308,0.674923,0.960181,B,0.655391,-0.051623,0.199056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
505,CALM1|KCNN4,CALM1,KCNN4,CALM1,KCNN4,3.560631,1.149412,0.000370,0.006200,NK,2.516959,0.526745,-0.385164,-25.897530,0.700116,0.969362,NK,0.000000,-0.183226,0.199056
506,PTPN6|CD300LF,PTPN6,CD300LF,PTPN6,CD300LF,-0.813125,-0.725591,0.416146,0.969362,NK,0.652221,-0.199005,-0.128388,-24.651196,0.897842,0.969362,NK,0.000000,-0.105409,0.199056
507,NUCB2|ERAP1,NUCB2,ERAP1,NUCB2,ERAP1,0.141227,0.482593,0.887691,0.969362,NK,0.156951,0.065837,0.654780,2.871128,0.512610,0.969362,NK,0.162055,0.282299,0.199056
508,SOCS2|EPOR,SOCS2,EPOR,SOCS2,EPOR,0.684737,1.819167,0.493510,0.969362,NK,0.230338,0.243138,-0.128388,-24.620756,0.897842,0.969362,NK,0.000000,-0.105409,0.199056


Recomplexify

In [36]:
# def recomplexify(lr_res, grps, complex_cols, complex_policy = 'min'):


# def _reduce_complexes():
    

In [37]:
complex_policy = 'min'

In [38]:
grps = ['source', 'target', 'ligand_complex', 'receptor_complex']

In [39]:
complex_cols = ['ligand_means' , 'receptor_means'] # specific for every method

In [40]:
add_cols = ['mat_mean', 'ligand', 'receptor']

In [41]:
# subset /w only means here - to be extended to all columns
lr_res = lr_res[ grps + complex_cols + add_cols ]

In [42]:
# temp = temp[[x=="INHBA_INHBB" for x in temp['ligand_complex']]].sort_values('receptor_complex')
# temp = temp[[x=="ACVR1B_ACVR2A" for x in temp['receptor_complex']]]

In [43]:
lr_res = lr_res.groupby(grps)

In [44]:
lr_res.obj

,source,target,ligand_complex,receptor_complex,ligand_means,receptor_means,mat_mean,ligand,receptor
0,B,B,LGALS9,PTPRC,0.374320,0.761533,0.199056,LGALS9,PTPRC
1,B,B,CD22,PTPRC,0.554265,0.761533,0.199056,CD22,PTPRC
2,B,B,LGALS1,PTPRC,0.611330,0.761533,0.199056,LGALS1,PTPRC
3,B,B,LGALS9,CD44,0.374320,0.655391,0.199056,LGALS9,CD44
4,B,B,ADAM10,CD44,0.119797,0.655391,0.199056,ADAM10,CD44
...,...,...,...,...,...,...,...,...,...
505,NK,NK,CALM1,KCNN4,2.516959,0.000000,0.199056,CALM1,KCNN4
506,NK,NK,PTPN6,CD300LF,0.652221,0.000000,0.199056,PTPN6,CD300LF
507,NK,NK,NUCB2,ERAP1,0.156951,0.162055,0.199056,NUCB2,ERAP1
508,NK,NK,SOCS2,EPOR,0.230338,0.000000,0.199056,SOCS2,EPOR


In [45]:
# Functions to be used to reduce the complexes
aggs = set([complex_policy, 'min']) # set to remove if both are min

In [46]:
complex_cols

['ligand_means', 'receptor_means']

In [47]:
cols_dict = {}

for col in complex_cols:
    cols_dict[col] = lr_res[col].agg(aggs).reset_index().copy().\
    rename(columns={agg:col.split('_')[0] + '_' + agg for agg in aggs})

In [48]:
lr_res = lr_res.obj.copy()

In [49]:
lr_res

,source,target,ligand_complex,receptor_complex,ligand_means,receptor_means,mat_mean,ligand,receptor
0,B,B,LGALS9,PTPRC,0.374320,0.761533,0.199056,LGALS9,PTPRC
1,B,B,CD22,PTPRC,0.554265,0.761533,0.199056,CD22,PTPRC
2,B,B,LGALS1,PTPRC,0.611330,0.761533,0.199056,LGALS1,PTPRC
3,B,B,LGALS9,CD44,0.374320,0.655391,0.199056,LGALS9,CD44
4,B,B,ADAM10,CD44,0.119797,0.655391,0.199056,ADAM10,CD44
...,...,...,...,...,...,...,...,...,...
505,NK,NK,CALM1,KCNN4,2.516959,0.000000,0.199056,CALM1,KCNN4
506,NK,NK,PTPN6,CD300LF,0.652221,0.000000,0.199056,PTPN6,CD300LF
507,NK,NK,NUCB2,ERAP1,0.156951,0.162055,0.199056,NUCB2,ERAP1
508,NK,NK,SOCS2,EPOR,0.230338,0.000000,0.199056,SOCS2,EPOR


In [50]:
for col in complex_cols:
    # left is lr_res /w the actual column name
    left_on = grps + [col]
    # right is the min subunit for that column
    join_key = col.split('_')[0] + '_min' # ligand_min or receptor_min
    right_on = grps + [join_key]
    
    # Here, I join the min value and keep only those rows that match
    lr_res = lr_res.merge(cols_dict[col], left_on=left_on, right_on=right_on).drop(join_key, 1)


<ipython-input-50-4e1c0206a125>:9: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  lr_res = lr_res.merge(cols_dict[col], left_on=left_on, right_on=right_on).drop(join_key, 1)


In [51]:
lr_res

,source,target,ligand_complex,receptor_complex,ligand_means,receptor_means,mat_mean,ligand,receptor
0,B,B,LGALS9,PTPRC,0.374320,0.761533,0.199056,LGALS9,PTPRC
1,B,B,CD22,PTPRC,0.554265,0.761533,0.199056,CD22,PTPRC
2,B,B,LGALS1,PTPRC,0.611330,0.761533,0.199056,LGALS1,PTPRC
3,B,B,LGALS9,CD44,0.374320,0.655391,0.199056,LGALS9,CD44
4,B,B,ADAM10,CD44,0.119797,0.655391,0.199056,ADAM10,CD44
...,...,...,...,...,...,...,...,...,...
4282,NK,NK,CALM1,KCNN4,2.516959,0.000000,0.199056,CALM1,KCNN4
4283,NK,NK,PTPN6,CD300LF,0.652221,0.000000,0.199056,PTPN6,CD300LF
4284,NK,NK,NUCB2,ERAP1,0.156951,0.162055,0.199056,NUCB2,ERAP1
4285,NK,NK,SOCS2,EPOR,0.230338,0.000000,0.199056,SOCS2,EPOR


In [ ]:
# More than those in LIANA - why? Duplicates?

CellPhoneDB re-implement

In [52]:
lr_res = lr_res.iloc[:, 0:9].copy()

In [53]:
lr_res['lr_mean'] = lr_res[['ligand_means', 'receptor_means']].mean(1)

In [54]:
lr_res

,source,target,ligand_complex,receptor_complex,ligand_means,receptor_means,mat_mean,ligand,receptor,lr_mean
0,B,B,LGALS9,PTPRC,0.374320,0.761533,0.199056,LGALS9,PTPRC,0.567927
1,B,B,CD22,PTPRC,0.554265,0.761533,0.199056,CD22,PTPRC,0.657899
2,B,B,LGALS1,PTPRC,0.611330,0.761533,0.199056,LGALS1,PTPRC,0.686432
3,B,B,LGALS9,CD44,0.374320,0.655391,0.199056,LGALS9,CD44,0.514855
4,B,B,ADAM10,CD44,0.119797,0.655391,0.199056,ADAM10,CD44,0.387594
...,...,...,...,...,...,...,...,...,...,...
4282,NK,NK,CALM1,KCNN4,2.516959,0.000000,0.199056,CALM1,KCNN4,1.258479
4283,NK,NK,PTPN6,CD300LF,0.652221,0.000000,0.199056,PTPN6,CD300LF,0.326110
4284,NK,NK,NUCB2,ERAP1,0.156951,0.162055,0.199056,NUCB2,ERAP1,0.159503
4285,NK,NK,SOCS2,EPOR,0.230338,0.000000,0.199056,SOCS2,EPOR,0.115169


Permutations

In [55]:
idx = np.arange(adata.X.shape[0])
rng = np.random.default_rng(seed=69)

In [56]:
n_perms = 1000

In [57]:
# Perm should be a cube of nperms x ngenes x #celltypes
perms = np.zeros((n_perms, labels.shape[0], adata.shape[1]))

In [58]:
labels_dict = {label:adata.obs.label.isin([label]) for label in labels}

In [59]:
labels[0]

'B'

In [60]:
# Assign Perms
for perm in range(n_perms):
    perm_idx = rng.permutation(idx)
    perm_mat = adata.X[perm_idx].copy()
    
    # populate matrix /w permuted means
    for cind in range(labels.shape[0]):
        perms[perm, cind] = perm_mat[labels_dict[labels[cind]]].mean(0)

In [61]:
# if working /w complexes
# ligand_pos = { entity:[ np.where(adata.var_names==x)[0][0] for x in entity.split('_') ] for entity in lr_res['ligand'] }
# receptor_pos = { entity:[ np.where(adata.var_names==x)[0][0] for x in entity.split('_') ] for entity in lr_res['receptor']}

In [62]:
ligand_pos = { entity:np.where(adata.var_names==entity)[0][0] for entity in lr_res['ligand'] }
receptor_pos = { entity:np.where(adata.var_names==entity)[0][0] for entity in lr_res['receptor'] }
labels_pos = { labels[pos]:pos for pos in range(labels.shape[0]) }

In [63]:
def _simple_mean(x, y): return(x + y)/2

def _build_ecdf(x):
    if((x.ligand_means==0) | (x.receptor_means==0)): return 1
    ligand_perms = perms[:, labels_pos[x.source], ligand_pos[x.ligand]]
    receptor_perms = perms[:, labels_pos[x.target], receptor_pos[x.receptor]]
    lr_means = _simple_mean(ligand_perms, receptor_perms)
    
    return (1 - ECDF(lr_means)(x.lr_mean))

In [64]:
lr_res

,source,target,ligand_complex,receptor_complex,ligand_means,receptor_means,mat_mean,ligand,receptor,lr_mean
0,B,B,LGALS9,PTPRC,0.374320,0.761533,0.199056,LGALS9,PTPRC,0.567927
1,B,B,CD22,PTPRC,0.554265,0.761533,0.199056,CD22,PTPRC,0.657899
2,B,B,LGALS1,PTPRC,0.611330,0.761533,0.199056,LGALS1,PTPRC,0.686432
3,B,B,LGALS9,CD44,0.374320,0.655391,0.199056,LGALS9,CD44,0.514855
4,B,B,ADAM10,CD44,0.119797,0.655391,0.199056,ADAM10,CD44,0.387594
...,...,...,...,...,...,...,...,...,...,...
4282,NK,NK,CALM1,KCNN4,2.516959,0.000000,0.199056,CALM1,KCNN4,1.258479
4283,NK,NK,PTPN6,CD300LF,0.652221,0.000000,0.199056,PTPN6,CD300LF,0.326110
4284,NK,NK,NUCB2,ERAP1,0.156951,0.162055,0.199056,NUCB2,ERAP1,0.159503
4285,NK,NK,SOCS2,EPOR,0.230338,0.000000,0.199056,SOCS2,EPOR,0.115169


In [65]:
lr_res['pvals'] = lr_res.apply(_build_ecdf, axis=1)

In [66]:
lr_res

,source,target,ligand_complex,receptor_complex,ligand_means,receptor_means,mat_mean,ligand,receptor,lr_mean,pvals
0,B,B,LGALS9,PTPRC,0.374320,0.761533,0.199056,LGALS9,PTPRC,0.567927,0.890
1,B,B,CD22,PTPRC,0.554265,0.761533,0.199056,CD22,PTPRC,0.657899,0.647
2,B,B,LGALS1,PTPRC,0.611330,0.761533,0.199056,LGALS1,PTPRC,0.686432,1.000
3,B,B,LGALS9,CD44,0.374320,0.655391,0.199056,LGALS9,CD44,0.514855,0.163
4,B,B,ADAM10,CD44,0.119797,0.655391,0.199056,ADAM10,CD44,0.387594,0.613
...,...,...,...,...,...,...,...,...,...,...,...
4282,NK,NK,CALM1,KCNN4,2.516959,0.000000,0.199056,CALM1,KCNN4,1.258479,1.000
4283,NK,NK,PTPN6,CD300LF,0.652221,0.000000,0.199056,PTPN6,CD300LF,0.326110,1.000
4284,NK,NK,NUCB2,ERAP1,0.156951,0.162055,0.199056,NUCB2,ERAP1,0.159503,0.066
4285,NK,NK,SOCS2,EPOR,0.230338,0.000000,0.199056,SOCS2,EPOR,0.115169,1.000


Calculate means_sums for NATMI

In [67]:
def _sum_means(lr_res, what, on):
    return lr_res.join(lr_res.groupby(on)[what].sum(), on=on, rsuffix='_sums')

In [68]:
lr_res = _sum_means(lr_res, what='ligand_means', on=['ligand_complex', 'receptor_complex', 'target'])
lr_res = _sum_means(lr_res, what='receptor_means', on=['ligand_complex', 'receptor_complex', 'source'])

In [69]:
lr_res

,source,target,ligand_complex,receptor_complex,ligand_means,receptor_means,mat_mean,ligand,receptor,lr_mean,pvals,ligand_means_sums,receptor_means_sums
0,B,B,LGALS9,PTPRC,0.374320,0.761533,0.199056,LGALS9,PTPRC,0.567927,0.890,0.534913,3.409275
1,B,B,CD22,PTPRC,0.554265,0.761533,0.199056,CD22,PTPRC,0.657899,0.647,0.672466,3.409275
2,B,B,LGALS1,PTPRC,0.611330,0.761533,0.199056,LGALS1,PTPRC,0.686432,1.000,3.588291,3.409275
3,B,B,LGALS9,CD44,0.374320,0.655391,0.199056,LGALS9,CD44,0.514855,0.163,0.534913,2.103060
4,B,B,ADAM10,CD44,0.119797,0.655391,0.199056,ADAM10,CD44,0.387594,0.613,0.362591,2.103060
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4282,NK,NK,CALM1,KCNN4,2.516959,0.000000,0.199056,CALM1,KCNN4,1.258479,1.000,6.145011,0.121264
4283,NK,NK,PTPN6,CD300LF,0.652221,0.000000,0.199056,PTPN6,CD300LF,0.326110,1.000,2.501501,0.052014
4284,NK,NK,NUCB2,ERAP1,0.156951,0.162055,0.199056,NUCB2,ERAP1,0.159503,0.066,0.386504,0.209576
4285,NK,NK,SOCS2,EPOR,0.230338,0.000000,0.199056,SOCS2,EPOR,0.115169,1.000,0.372007,0.050942


NATMI fun

In [ ]:
def _natmi_score(x):
    lig = (x.ligand_means / x.ligand_means_sums)
    rec = (x.receptor_means / x.receptor_means_sums)
    return  lig * rec

In [ ]:
lr_res['edge_specificity'] = lr_res.apply(_natmi_score, axis=1)

In [ ]:
lr_res.sort_values('edge_specificity', ascending=False)

SCA re-implement

In [ ]:
def _sca_score(x):
    lr_sqrt = np.sqrt(x.ligand_means) * np.sqrt(x.receptor_means)
    return lr_sqrt / (lr_sqrt + x.mat_mean)

In [ ]:
lr_res['lrscore'] = lr_res.apply(_sca_score, axis=1)

In [ ]:
lr_res.sort_values('lrscore', ascending=False)

In [ ]:
lr_res[(lr_res.ligand_complex=='TGFB1') & (lr_res.receptor_complex=='ACVR1_TGFBR1_TGFBR2')].sort_values('lrscore', ascending=False)

logFC re-implement

In [ ]:
lr_res['logfc'] = lr_res[['ligand_logfoldchanges', 'receptor_logfoldchanges']].mean(1)

In [ ]:
lr_res.sort_values(by='logfc', key=abs, ascending=False)

Re-implement Connectome

In [ ]:
lr_res['edge_weight'] = lr_res[['ligand_zscores', 'receptor_zscores']].mean(1)